### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [7]:
# Calculate local government area metrics
total_unique_schools = school_data['school_name'].nunique()
total_students = student_data['student_name'].count()
total_area_budget = school_data['budget'].sum()
avg_math_score = student_data['maths_score'].mean()  # Change 'math_score' to 'maths_score'
avg_reading_score = student_data['reading_score'].mean()

# Calculate passing percentages in the local government area
passing_math = student_data[student_data['maths_score'] >= 50]  # Change 'math_score' to 'maths_score'
passing_reading = student_data[student_data['reading_score'] >= 50]

percentage_passing_math = (passing_math['student_name'].count() / total_students) * 100
percentage_passing_reading = (passing_reading['student_name'].count() / total_students) * 100

# Calculate overall passing rate in the local government area
overall_passing_rate = (percentage_passing_math + percentage_passing_reading) / 2

# Create a DataFrame for local government area metrics
area_metrics_df = pd.DataFrame({
    'Total Unique Schools': [total_unique_schools],
    'Total Students': [total_students],
    'Total Area Budget': [total_area_budget],
    'Average Math Score': [avg_math_score],
    'Average Reading Score': [avg_reading_score],
    '% Passing Math': [percentage_passing_math],
    '% Passing Reading': [percentage_passing_reading],
    '% Overall Passing Rate': [overall_passing_rate]
})

# Format the Total Area Budget column as currency
area_metrics_df['Total Area Budget'] = area_metrics_df['Total Area Budget'].map('${:,.2f}'.format)
area_metrics_df


,Total Unique Schools,Total Students,Total Area Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",70.338192,69.980138,86.078632,84.426857,85.252744


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [48]:
import pandas as pd

# Assuming you have DataFrames named student_data and school_data
# ...

# Calculate total students, average math score, and average reading score for each school
school_summary = student_data.groupby('school_name').agg({
    'Student ID': 'count',
    'maths_score': 'mean',
    'reading_score': 'mean'
}).reset_index()

# Calculate total school budget for each school
total_budget = school_data.groupby('school_name')['budget'].sum().reset_index()

# Merge total budget into the school_summary DataFrame
school_summary = pd.merge(school_summary, total_budget, on='school_name', how='left')

# Add the 'type' column to school_summary by merging it from school_data
school_type = school_data[['school_name', 'type']].drop_duplicates()
school_summary = pd.merge(school_summary, school_type, on='school_name', how='left')

# Calculate per student budget
school_summary['Per Student Budget'] = school_summary['budget'] / school_summary['Student ID']

# Calculate the percentage of students passing math (score >= 50)
passing_math = student_data[student_data['maths_score'] >= 50].groupby('school_name')['Student ID'].count().reset_index()
passing_math.rename(columns={'Student ID': 'Passing Math'}, inplace=True)

# Calculate the percentage of students passing reading (score >= 50)
passing_reading = student_data[student_data['reading_score'] >= 50].groupby('school_name')['Student ID'].count().reset_index()
passing_reading.rename(columns={'Student ID': 'Passing Reading'}, inplace=True)

# Calculate the percentage of students passing both math and reading
passing_both = student_data[(student_data['maths_score'] >= 50) & (student_data['reading_score'] >= 50)].groupby('school_name')['Student ID'].count().reset_index()
passing_both.rename(columns={'Student ID': 'Overall Passing'}, inplace=True)

# Merge passing percentages into the school_summary DataFrame
school_summary = pd.merge(school_summary, passing_math, on='school_name', how='left')
school_summary = pd.merge(school_summary, passing_reading, on='school_name', how='left')
school_summary = pd.merge(school_summary, passing_both, on='school_name', how='left')

# Calculate the percentage of passing math, passing reading, and overall passing
school_summary['% Passing Math'] = (school_summary['Passing Math'] / school_summary['Student ID']) * 100
school_summary['% Passing Reading'] = (school_summary['Passing Reading'] / school_summary['Student ID']) * 100
school_summary['% Overall Passing'] = (school_summary['Overall Passing'] / school_summary['Student ID']) * 100

# Set "school_name" as the index
school_summary.set_index('school_name', inplace=True)

# Reorder columns
school_summary = school_summary[['type', 'Student ID', 'budget', 'Per Student Budget',
                                 'maths_score', 'reading_score', '% Passing Math', '% Passing Reading', '% Overall Passing']]

# Rename columns for clarity
school_summary.rename(columns={
    'type': 'School Type',
    'Student ID': 'Total Students',
    'budget': 'Total School Budget',
    'maths_score': 'Average Math Score',
    'reading_score': 'Average Reading Score'
}, inplace=True)

# Format the Total School Budget and Per Student Budget columns as currency
school_summary['Total School Budget'] = school_summary['Total School Budget'].map('${:,.2f}'.format)
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map('${:,.2f}'.format)

# Display the School Summary DataFrame
school_summary



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [47]:
import pandas as pd

# Assuming you have a DataFrame named school_summary with relevant data

# Sort the DataFrame by '% Overall Passing' in descending order and select the top 5 rows
top_performing_schools = school_summary.sort_values(by='% Overall Passing', ascending=False).head(5)

# Display the top-performing schools
top_performing_schools



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [46]:
import pandas as pd

# Assuming you have a DataFrame named school_summary with relevant data

# Sort the DataFrame by '% Overall Passing' in ascending order and select the first 5 rows
bottom_performing_schools = school_summary.sort_values('% Overall Passing').head(5)

# Display the bottom-performing schools
bottom_performing_schools


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [50]:
# Create pandas series for each year level (math scores)
math_scores_9th = student_data[student_data['year'] == 9].groupby('school_name')['maths_score'].mean()
math_scores_10th = student_data[student_data['year'] == 10].groupby('school_name')['maths_score'].mean()
math_scores_11th = student_data[student_data['year'] == 11].groupby('school_name')['maths_score'].mean()
math_scores_12th = student_data[student_data['year'] == 12].groupby('school_name')['maths_score'].mean()

# Combine the series into a DataFrame
math_scores_by_year = pd.DataFrame({
    'School Name': math_scores_9th.index,  # Add this line to make "School Name" the first column
    '9th Grade Math Score': math_scores_9th,
    '10th Grade Math Score': math_scores_10th,
    '11th Grade Math Score': math_scores_11th,
    '12th Grade Math Score': math_scores_12th
})

# Optional: Format the displayed data for cleaner formatting
math_scores_by_year = math_scores_by_year.set_index('School Name')  # Set "School Name" as the index

# Display the DataFrame
math_scores_by_year

,9th Grade Math Score,10th Grade Math Score,11th Grade Math Score,12th Grade Math Score
School Name,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393


## Reading Score by Year

* Perform the same operations as above for reading scores

In [49]:
# Create pandas series for each year level (reading scores)
reading_scores_9th = student_data[student_data['year'] == 9].groupby('school_name')['reading_score'].mean()
reading_scores_10th = student_data[student_data['year'] == 10].groupby('school_name')['reading_score'].mean()
reading_scores_11th = student_data[student_data['year'] == 11].groupby('school_name')['reading_score'].mean()
reading_scores_12th = student_data[student_data['year'] == 12].groupby('school_name')['reading_score'].mean()

# Combine the series into a DataFrame
reading_scores_by_year = pd.DataFrame({
    'School Name': reading_scores_9th.index,  # Add this line to make "School Name" the first column
    '9th Grade Reading Score': reading_scores_9th,
    '10th Grade Reading Score': reading_scores_10th,
    '11th Grade Reading Score': reading_scores_11th,
    '12th Grade Reading Score': reading_scores_12th
})

# Optional: Format the displayed data for cleaner formatting
reading_scores_by_year = reading_scores_by_year.set_index('School Name')  # Set "School Name" as the index

# Display the DataFrame
reading_scores_by_year

,9th Grade Reading Score,10th Grade Reading Score,11th Grade Reading Score,12th Grade Reading Score
School Name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [41]:
# Calculate per student budget for each school
school_data['Per Student Budget'] = school_data['budget'] / school_data['size']

# Define spending ranges (bins)
spending_bins = [0, 585, 630, 645, 680]  # You can adjust these bin values as needed
spending_labels = ['<$585', '$585-630', '$630-645', '$645-680']

# Assign each school to a spending range
school_data['Spending Range (Per Student)'] = pd.cut(school_data['Per Student Budget'], spending_bins, labels=spending_labels, include_lowest=True)

# Merge school_data with student_data
merged_data = student_data.merge(school_data, on='school_name')

# Calculate average math and reading scores for each spending range
spending_math_scores = merged_data.groupby('Spending Range (Per Student)')['maths_score'].mean()
spending_reading_scores = merged_data.groupby('Spending Range (Per Student)')['reading_score'].mean()

# Calculate the percentage of students passing math for each spending range
passing_math_df = merged_data[merged_data['maths_score'] >= 50]
passing_math_percent = passing_math_df.groupby('Spending Range (Per Student)')['maths_score'].count() / merged_data.groupby('Spending Range (Per Student)')['maths_score'].count() * 100

# Calculate the percentage of students passing reading for each spending range
passing_reading_df = merged_data[merged_data['reading_score'] >= 50]
passing_reading_percent = passing_reading_df.groupby('Spending Range (Per Student)')['reading_score'].count() / merged_data.groupby('Spending Range (Per Student)')['reading_score'].count() * 100

# Calculate the overall passing rate for each spending range
overall_passing_rate = (passing_math_percent + passing_reading_percent) / 2

# Create a DataFrame to display the results
spending_summary = pd.DataFrame({
    'Average Math Score': spending_math_scores,
    'Average Reading Score': spending_reading_scores,
    '% Passing Math': passing_math_percent,
    '% Passing Reading': passing_reading_percent,
    '% Overall Passing Rate': overall_passing_rate
})

# Optional: Format the displayed data for cleaner formatting
spending_summary.index.name = 'Spending Range (Per Student)'

# Display the DataFrame
spending_summary



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Range (Per Student),,,,,
<$585,70.938128,70.379397,88.159548,85.568467,86.864008
$585-630,72.173448,70.965856,91.556671,87.345915,89.451293
$630-645,70.104045,69.953630,85.391274,84.260731,84.826002
$645-680,68.876878,69.064160,81.556079,81.815967,81.686023


## Scores by School Size

* Perform the same operations as above, based on school size.

In [40]:
# Define school size bins
size_bins = [0, 1000, 2000, 5000]  # You can adjust these bin values as needed
size_labels = ['Small (<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']

# Assign each school to a size range
school_data['School Size'] = pd.cut(school_data['size'], size_bins, labels=size_labels, include_lowest=True)

# Merge school_data with student_data
merged_data = student_data.merge(school_data, on='school_name')

# Calculate average math and reading scores for each size range
size_math_scores = merged_data.groupby('School Size')['maths_score'].mean()
size_reading_scores = merged_data.groupby('School Size')['reading_score'].mean()

# Calculate the percentage of students passing math for each size range
passing_math_df = merged_data[merged_data['maths_score'] >= 50]
passing_math_percent = passing_math_df.groupby('School Size')['maths_score'].count() / merged_data.groupby('School Size')['maths_score'].count() * 100

# Calculate the percentage of students passing reading for each size range
passing_reading_df = merged_data[merged_data['reading_score'] >= 50]
passing_reading_percent = passing_reading_df.groupby('School Size')['reading_score'].count() / merged_data.groupby('School Size')['reading_score'].count() * 100

# Calculate the overall passing rate for each size range
overall_passing_rate = (passing_math_percent + passing_reading_percent) / 2

# Create a DataFrame to display the results
size_summary = pd.DataFrame({
    'Average Math Score': size_math_scores,
    'Average Reading Score': size_reading_scores,
    '% Passing Math': passing_math_percent,
    '% Passing Reading': passing_reading_percent,
    '% Overall Passing Rate': overall_passing_rate
})

# Optional: Format the displayed data for cleaner formatting
size_summary.index.name = 'School Size'

# Display the DataFrame
size_summary



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),72.240461,71.627790,91.144708,87.185025,89.164867
Medium (1000-2000),71.441798,70.724595,89.908472,86.740202,88.324337
Large (2000-5000),69.926450,69.685088,84.722649,83.622133,84.172391


## Scores by School Type

* Perform the same operations as above, based on school type

In [39]:
# Merge school_data with student_data
merged_data = student_data.merge(school_data, on='school_name')

# Calculate average math and reading scores for each school type
type_math_scores = merged_data.groupby('type')['maths_score'].mean()
type_reading_scores = merged_data.groupby('type')['reading_score'].mean()

# Calculate the percentage of students passing math for each school type
passing_math_df = merged_data[merged_data['maths_score'] >= 50]
passing_math_percent = passing_math_df.groupby('type')['maths_score'].count() / merged_data.groupby('type')['maths_score'].count() * 100

# Calculate the percentage of students passing reading for each school type
passing_reading_df = merged_data[merged_data['reading_score'] >= 50]
passing_reading_percent = passing_reading_df.groupby('type')['reading_score'].count() / merged_data.groupby('type')['reading_score'].count() * 100

# Calculate the overall passing rate for each school type
overall_passing_rate = (passing_math_percent + passing_reading_percent) / 2

# Create a DataFrame to display the results
type_summary = pd.DataFrame({
    'Average Math Score': type_math_scores,
    'Average Reading Score': type_reading_scores,
    '% Passing Math': passing_math_percent,
    '% Passing Reading': passing_reading_percent,
    '% Overall Passing Rate': overall_passing_rate
})

# Optional: Format the displayed data for cleaner formatting
type_summary.index.name = 'School Type'

# Display the DataFrame
type_summary



,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Government,69.990399,69.753485,84.886566,83.818950,84.352758
Independent,71.107594,70.481548,88.715762,85.771691,87.243726
